# Reading and writing files

This tutorial includes an overview of the different ways available to load the binary arrays from the disc after running a numerical simulation with [XCompact3d](https://github.com/xcompact3d/Incompact3d).
Besides that, some options are presented to save the results from our analysis, together with some tips and tricks.

<div class="alert alert-info">

For an interactive experience [launch this tutorial on Binder](https://mybinder.org/v2/gh/fschuch/xcompact3d_toolbox/main?labpath=lab%2Ftree%2Fdocs%2Ftutorial).

</div>

## Preparation

Here we prepare the dataset for this notebook, so it can be reproduced on local machines or on the cloud, you are invited to test and interact with many of the concepts.
It also provides nice support for courses and tutorials, let us know if you produce any of them.

The very first step is to import the toolbox and other packages:

In [ ]:
import warnings

import numpy as np
import xarray as xr
import xcompact3d_toolbox as x3d

Then we can download an example from the [online database](https://github.com/fschuch/xcompact3d_toolbox_data), the flow around a cylinder in this case.
We set `cache=True` and a local destination where it can be saved in our computer `cache_dir="./example/"`, so there is no need to download it everytime the kernel is restarted.

In [ ]:
cylinder_ds, prm = x3d.tutorial.open_dataset(
    "cylinder", cache=True, cache_dir="./example/"
)

let's take a look at the dataset:

In [ ]:
cylinder_ds.info()

We got a [xarray.Dataset](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.html#xarray.Dataset) with the variables `u` (velocity vector), `pp` (pressure) and `epsi` (describes the geometry), their coordinates (`x`, `y`, `t` and `i`) and some atributes like the `xcompact3d_version` used to run this simulation, the `url` where you can find the dataset, and others.

In the next block, we configure the toolbox and some atributes at the dataset, so we can write all the binary fields to the disc.
Do not worry about the details right now, this is just the preparation step, we are going to discuss them later.

In [ ]:
x3d.param["mytype"] = np.float32

prm.dataset.set(data_path="./data/", drop_coords="z")

cylinder_ds.u.attrs["file_name"] = "u"
cylinder_ds.pp.attrs["file_name"] = "pp"
cylinder_ds.epsi.attrs["file_name"] = "epsilon"

prm.write("input.i3d")

prm.dataset.write(cylinder_ds)

prm.dataset.write_xdmf("xy-planes.xdmf")

del cylinder_ds, prm

After that, the files are organized as follow:

```
tutorial
│   computing_and_plotting.ipynb
│   io.ipynb
│   input.i3d
│   parameters.ipynb
│   xy-planes.xdmf
│
└─── data
│       │   epsilon.bin
│       │   pp-000.bin
│       │   pp-001.bin
│       │   ... 
│       │   pp-199.bin
│       │   pp-200.bin
│       │   ux-000.bin
│       │   ux-001.bin
│       │   ... 
│       │   ux-199.bin
│       │   ux-200.bin
│       │   uy-000.bin
│       │   uy-001.bin
│       │   ... 
│       │   uy-199.bin
│       │   uy-200.bin
│       │   uz-000.bin
│       │   uz-001.bin
│       │   ... 
│       │   uz-199.bin
│       │   uz-200.bin
│
└─── example
│       │   cylinder.nc
```

It is very similar to what we get after successfully running a simulation, so now we can move on to the tutorial.

## Why xarray?

The data structures are provided by [xarray](http://xarray.pydata.org/en/stable/index.html), that introduces labels in the form of dimensions, coordinates and attributes on top of raw NumPy-like arrays, which allows for a more intuitive, more concise, and less error-prone developer experience.
It integrates tightly with [dask](https://dask.org/) for parallel computing.

The goal here is to speed up the development of customized post-processing applications with the concise interface provided by [xarray](http://xarray.pydata.org/en/stable/index.html). Ultimately, we can compute solutions with fewer lines of code and better readability, so we expend less time testing and debugging and more time exploring our datasets and getting insights.

Additionally, xcompact3d-toolbox includes extra functionalities for [DataArray](https://xcompact3d-toolbox.readthedocs.io/en/stable/Docstrings.html#xcompact3d_toolbox.array.X3dDataArray) and [Dataset](https://xcompact3d-toolbox.readthedocs.io/en/stable/Docstrings.html#xcompact3d_toolbox.array.X3dDataset).

Before going forward, please, take a look at [Overview: Why xarray?](http://xarray.pydata.org/en/stable/getting-started-guide/why-xarray.html) and [Quick overview](http://xarray.pydata.org/en/stable/getting-started-guide/quick-overview.html) to understand the motivation to use [xarray](http://xarray.pydata.org/en/stable/index.html)'s data structures instead of just numpy-like arrays.

## Xarray objects on demand

To start our post-processing, let's load the parameters file:

In [ ]:
prm = x3d.Parameters(loadfile="input.i3d")

Notice there is an entire [tutorial dedicated to it](https://xcompact3d-toolbox.readthedocs.io/en/stable/tutorial/parameters.html).

To save space on the disc, our dataset was converted from double precision to single, so we have to configure the toolbox to:

In [ ]:
x3d.param["mytype"] = np.float32

The methods in the toolbox support different [filename properties](https://xcompact3d-toolbox.readthedocs.io/en/stable/Docstrings.html#xcompact3d_toolbox.io.FilenameProperties), like the classic `ux000` or the new `ux-0000.bin`, besides some combinations between them. For our case, we set the parameters as:

In [ ]:
prm.dataset.filename_properties.set(
    separator = "-",
    file_extension = ".bin",
    number_of_digits = 3,
)

Now we specify the parameters for our dataset, like where it is found (`data_path`), if it needs to drop some coordinate (`drop_coords`, again, to save space, we are working with a span-wise averaged dataset, so we drop `z` to work with `xy` planes), we inform the parameter that controls the number of timesteps `snapshot_counting` and their step `snapshot_step`.
Consult the [dataset documentation](https://xcompact3d-toolbox.readthedocs.io/en/stable/Docstrings.html#xcompact3d_toolbox.io.Dataset) to see different ways to customize your experience, and choose the ones that best suits your post-processing application.
In this example, they are defined as:

In [ ]:
prm.dataset.set(
    data_path="./data/",
    drop_coords="z",
    snapshot_counting="ilast",
    snapshot_step="ioutput"
)

Now we are good to go.

We can check the [length of the dataset](https://xcompact3d-toolbox.readthedocs.io/en/stable/Docstrings.html#xcompact3d_toolbox.io.Dataset.__len__) we are dealing with:

In [ ]:
len(prm.dataset)

Meaning that our binary files range from 0 (i.g., `ux-000.bin`) to 200 (i.g., `ux-200.bin`), exactly as expected.

It is possible to load any given array:

In [ ]:
epsilon = prm.dataset.load_array("./data/epsilon.bin", add_time=False)

Notice that [load_array](https://xcompact3d-toolbox.readthedocs.io/en/stable/Docstrings.html#xcompact3d_toolbox.io.Dataset.load_array) requires the entire path to the file, and we use `add_time=False` because this array does not evolve in time like the others, i.e., it is not numerated for several snapshots.

We can see it on the screen:

In [ ]:
epsilon

Let's do it again, this time for `ux` and using `add_time=True`:

In [ ]:
ux = prm.dataset.load_array("./data/ux-100.bin", add_time=True)

See that `t` is now a coordinate, and for this snapshot it was computed automatically as dimensionless time `75.0`:

In [ ]:
ux

That is not all. If you have enough memory, you can load the entire time series for a given variable with [load_time_series](https://xcompact3d-toolbox.readthedocs.io/en/stable/Docstrings.html#xcompact3d_toolbox.io.Dataset.load_time_series), or simply by:

In [ ]:
ux = prm.dataset["ux"]

Let's see it (note 201 files are loaded and wrapped with the appropriate coordinates):

In [ ]:
ux

You can store each array in a different variable, like:

In [ ]:
ux = prm.dataset["ux"]
uy = prm.dataset["uy"]
pp = prm.dataset["pp"]

Or organize many arrays in a dataset:

In [ ]:
# create an empty dataset
ds = xr.Dataset()

# populate it
for var in ["ux", "uy", "pp"]:
    ds[var] = prm.dataset[var]

# show on the screen
ds

It is possible to load all the variables from a given snapshot with [load_snapshot](https://xcompact3d-toolbox.readthedocs.io/en/stable/Docstrings.html#xcompact3d_toolbox.io.Dataset.load_snapshot), or simply:

In [ ]:
snapshot = prm.dataset[100]

And we got a [xarray.Dataset](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.html#xarray.Dataset) with all the variables and their coordinates. You can access each of them with the dot notation (i.g., `snapshot.pp`, `snapshot.ux`, `snapshot.uy`) or the dict-like notation (i.g., `snapshot["pp"]`, `snapshot["ux"]`, `snapshot["uy"]`). See the dataset:

In [ ]:
snapshot

Do you need the snapshots in a range? No problem. Let's do a slice to load the last 100, and just to exemplify, compute a time average:

In [ ]:
time_averaged = prm.dataset[-100:].mean("t")
time_averaged

You can even use the slice notation to load all the snapshots at once:

In [ ]:
prm.dataset[:]

Of course, some simulations may not fit in the memory like in this tutorial. For these cases we can iterate over all snapshots, loading them one by one:

In [ ]:
for ds in prm.dataset:
    # Computing the vorticity, just to exemplify
    vort = ds.uy.x3d.first_derivative("x") - ds.ux.x3d.first_derivative("y")

Note that `reversed(prm.dataset)` also works.

Or for better control, we can iterate over a selected range of snapshots loading them one by one. The arguments are the same of a classic [range](https://docs.python.org/3/library/functions.html#func-range) in Python:

In [ ]:
for ds in prm.dataset(100, 200, 1):
    # Computing the vorticity, just to exemplify
    vort = ds.uy.x3d.first_derivative("x") - ds.ux.x3d.first_derivative("y")

In [ ]:
# Result from the last iteration
vort

## Writting the results to binary files

In the last example we computed the vorticity but did nothing with it. This time, let's write it to the disc using [write](https://xcompact3d-toolbox.readthedocs.io/en/stable/Docstrings.html#xcompact3d_toolbox.io.Dataset.write):

In [ ]:
for ds in prm.dataset:
    vort = ds.uy.x3d.first_derivative("x") - ds.ux.x3d.first_derivative("y")
    prm.dataset.write(data = vort, file_prefix = "w3")

The example above works for a [xarray.DataArray](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.html#xarray.DataArray). We can do it for a [xarray.Dataset](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.html#xarray.Dataset) as well, but with one key difference. Only the arrays with an attribute called `file_name` will be written. It is done to avoid overwriting the base field (`ux`, `uy`, `uz`, ...) by accident.

Let's rewrite the previous example to store `vort` in the dataset `ds`. We set an atribute `file_name` to `w3`, so the arrays will be written as `w3-000.bin`, `w3-001.bin`, `w3-002.bin`, etc.

We are also suppressing warnings, because the application will tell us it can not save `pp`, `ux` and `uy`, since they do not have a `file_name`. But in fact, we do not want to rewrite them anyway.

See the code:

In [ ]:
with warnings.catch_warnings():
    warnings.filterwarnings('ignore', category=UserWarning)
    for ds in prm.dataset:
        ds["vort"] = ds.uy.x3d.first_derivative("x") - ds.ux.x3d.first_derivative("y")
        ds["vort"].attrs["file_name"] = "w3"
        prm.dataset.write(ds)

See the dataset from the last iteration:

In [ ]:
ds

The method [prm.dataset.write()](https://xcompact3d-toolbox.readthedocs.io/en/stable/Docstrings.html#xcompact3d_toolbox.io.Dataset.write) writes the files as raw binaries in the same way that [XCompact3d](https://github.com/xcompact3d/Incompact3d) would do. It means you can read them at the flow solver and also process them on any other tool that you are already familiar with, including the toolbox.

For instance, we get `w3` if we load snapshot 0 again:

In [ ]:
prm.dataset[0]

### Update the xdmf file

After computing and writing new results to the disc, you can to open them on any external tools, like Paraview or Visit. You can update the xdmf file to include the recently computed `w3`. See the code:

In [ ]:
prm.dataset.write_xdmf("xy-planes.xdmf")

## Other formats

Xarray objects can be exported to many other formats, depending on your needs.

For instance, [xarray.DataArray](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.html#xarray.DataArray) and [xarray.Dataset](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.html#xarray.Dataset) can be written as [netCDF](http://xarray.pydata.org/en/stable/user-guide/io.html). In this way, they will keep all dimensions, coordinates, and attributes. This format is easier to handle and share because the files are self-sufficient. It is the format used to download the dataset used in this tutorial, and it is a good alternative to use when sharing the results of your research.

Just to give you an estimation about the disk usage, the size of the dataset `cylinder.nc` that we downloaded for this tutorial is 75.8 MB. The size of the folder `./data/` after producing the binary files in the same way that [XCompact3d](https://github.com/xcompact3d/Incompact3d) would do is 75.7 MB.

To exemplify the use of netCDF, let's take one snapshot:

In [ ]:
snapshot = prm.dataset[0]
snapshot

Now, let's include additional information for the ones that are going to use our data. You can set attributes for each array, coordinate, and also global attributes for the dataset. They are stored in a dictionary.

See the example:

In [ ]:
# Setting attributes for each coordinate
snapshot.x.attrs = dict(
    name = "x",
    long_name = "Stream-wise coordinate",
    units = "-"
)
snapshot.y.attrs = dict(
    name = "y",
    long_name = "Vertical coordinate",
    units = "-"
)
snapshot.t.attrs = dict(
    name = "t",
    long_name = "Time",
    units = "-"
)
# Setting attributes for each array
snapshot.ux.attrs = dict(
    name = "ux",
    long_name = "Stream-wise velocity",
    units = "-"
)
snapshot.uy.attrs = dict(
    name = "y",
    long_name = "Vertical velocity",
    units = "-"
)
snapshot.pp.attrs = dict(
    name = "p",
    long_name = "Pressure",
    units = "-"
)
snapshot.w3.attrs = dict(
    name = "w3",
    long_name = "Vorticity",
    units = "-"
)
# Setting attributes for the dataset
snapshot.attrs = dict(
    title = "An example from the tutorials",
    url = "https://xcompact3d-toolbox.readthedocs.io/en/stable/tutorial/io.html",
    authors = "List of names",
    doi = "maybe a fancy doi from zenodo",
)

Exporting it as a netCDF file:

In [ ]:
snapshot.to_netcdf("snapshot-000.nc")

Importing the netCDF file:

In [ ]:
snapshot_in = xr.open_dataset("snapshot-000.nc")

See the result, it keeps all dimensions, coordinates, and attributes:

In [ ]:
snapshot_in

If you manage to reduce the dataset's dimensions with some integral, average, or selecting subsets of data, you can convert it to a `pandas.Dataframe` and then export it to CSV, Excel, and many other options.

For instance, let's select a vertical profile for all variables where `x = 20` and convert it to a dataframe:

In [ ]:
snapshot_in.sel(x=20.0).to_dataframe()

Now, you can refer to [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/index.html) for more details.